# Creating the data to test in STATA and Python

In [14]:
import warnings
warnings.filterwarnings("ignore") # I suppressed the warnings to make it easier to read, we should still try to fix them
import numpy as np
import pandas as pd
from ujive1 import *
from ujive2 import *
from tsls import * 

#Pick a vector length:
n = 1000

#Getting our Z's and making a Z matrix:
Z = np.random.randn(n, 1)
column_of_ones = np.ones((Z.shape[0], 1))
Z = np.hstack((column_of_ones, Z))
#pprint(Z)

#Parameter vectors:
α = np.array([1,1])
β = np.array([1,2])
#pprint(α)
#pprint(β)

#Error terms:
e1 = np.random.normal(0,5,n)
e2 = np.random.normal(0,5,n)
δ = np.random.normal(0,1)
ε = 5*e1 - 5*e2 + δ

#Making our endogenous variable:
x = np.dot(Z,α) + .2*e1
X = np.column_stack((column_of_ones, x))
#pprint(X)

#Outcome vector:
Y = np.dot(X,β) + ε

#OLS benchmark:
bhat_ols = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T, Y))

# Check to see if the Z'Z matrix is invertible
if np.linalg.matrix_rank(Z.T @ Z) == Z.shape[1]:  # Should be True
    print("Z'Z is invertible")
else:
    print("Z'Z is not invertible")

cond_number = np.linalg.cond(Z.T @ Z)
print(f"Condition number of Z.T @ Z: {cond_number}")
if cond_number > 1e10:  # Threshold for ill-conditioning
    raise ValueError("Z.T @ Z is ill-conditioned and may cause numerical instability.")


#2sls comparison:
Zt_Z = np.dot(Z.T, Z)
Zt_Z_inv = np.linalg.inv(Zt_Z)
pz = np.dot(np.dot(Z, Zt_Z_inv), Z.T)
proj_x = np.dot(pz, X)
first = np.linalg.inv(np.dot(proj_x.T, X))
second = np.dot(proj_x.T, Y)
bhat_2sls = np.dot(first, second)


ujive1 = UJIVE1(Y,X,Z,talk=False)
ujive2 = UJIVE2(Y,X,Z,talk=False)
#tsls = TSLS(Y,X,Z)

# Combine matrices into a single DataFrame
df = pd.DataFrame({
    "Y": Y,  # Outcome vector
    **{f"X{i}": X[:, i] for i in range(X.shape[1])},  # Endogenous variables
    **{f"Z{i}": Z[:, i] for i in range(Z.shape[1])}   # Instrumental variables
})

# Save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)

#Compare them:
print("OLS:", bhat_ols[1])
print("2SLS:", bhat_2sls[1])
#print("Jive 1:", ujive1['beta'])
print(ujive1.summary())
#print("Jive 2:",ujive2['beta'])
print(ujive2.summary())

Normally this estimator is used when Z has more columns than X. In this case Z has 2 columns and X has 2 columns.


Z'Z is invertible
Condition number of Z.T @ Z: 1.0500438905905647
OLS: 14.041014363509797
2SLS: 1.1892218628395628

UJIVE1 Regression Results
 Coefficient  Std. Error   t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    1.068765    1.571247 0.680202 0.496534       -2.014561         4.152092
    1.136565    1.163602 0.680202 0.496534       -1.146822         3.419952
--------------------------------------------------------------------------------
R-squared: 0.045181
Adjusted R-squared: 0.044224
F-statistic: 1.991932
Root MSE: 35.546944
None

UJIVE2 Regression Results
 Coefficient  Std. Error  t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    1.064228    1.571281  0.6773 0.498373       -2.019166         4.147622
    1.141142    1.163826  0.6773 0.498373       -1.142685         3.424969
--------------------------------------------------------------------------------
R-squared: 0.045355
Adjusted R-squared: 0.044399
F-statistic: 2.008375
Root MSE: 35.543700
None


In [7]:
(Z @ np.linalg.inv(np.dot(Z.T,Z)))

array([[ 0.00100136, -0.00104589],
       [ 0.0010009 , -0.00069287],
       [ 0.00099984,  0.00012111],
       ...,
       [ 0.00100183, -0.00140459],
       [ 0.00099925,  0.00057684],
       [ 0.00100163, -0.00125753]], shape=(1000, 2))

In [9]:
(Z @ np.linalg.pinv(np.dot(Z.T,Z))) @ Z.T

/var/folders/56/826nnkqj62x3f69z9wt21fmr0000gn/T/ipykernel_3379/1661977985.py:1: RuntimeWarning: divide by zero encountered in matmul
  (Z @ np.linalg.pinv(np.dot(Z.T,Z))) @ Z.T
/var/folders/56/826nnkqj62x3f69z9wt21fmr0000gn/T/ipykernel_3379/1661977985.py:1: RuntimeWarning: overflow encountered in matmul
  (Z @ np.linalg.pinv(np.dot(Z.T,Z))) @ Z.T
/var/folders/56/826nnkqj62x3f69z9wt21fmr0000gn/T/ipykernel_3379/1661977985.py:1: RuntimeWarning: invalid value encountered in matmul
  (Z @ np.linalg.pinv(np.dot(Z.T,Z))) @ Z.T


array([[0.0021327 , 0.00175037, 0.00086884, ..., 0.00252116, 0.00037529,
        0.0023619 ],
       [0.00175037, 0.00149709, 0.00091311, ..., 0.00200772, 0.00058615,
        0.00190221],
       [0.00086884, 0.00091311, 0.00101519, ..., 0.00082386, 0.00107234,
        0.0008423 ],
       ...,
       [0.00252116, 0.00200772, 0.00082386, ..., 0.00304285, 0.00016104,
        0.00282897],
       [0.00037529, 0.00058615, 0.00107234, ..., 0.00016104, 0.00134455,
        0.00024888],
       [0.0023619 , 0.00190221, 0.0008423 , ..., 0.00282897, 0.00024888,
        0.00263748]], shape=(1000, 1000))